In [12]:
import numpy as np
import itertools as iter

# Часть 1.

In [13]:
# Начальная матрица
S = np.array([
    [1, 0, 0, 1, 0, 1, 1],
    [1, 1, 0, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 1, 1, 1, 0]
])

## Сформировать порождающую матрицу линейного кода (7, 4, 3).

In [14]:
# Выполняет операцию XOR
def elements_row_xor(row1, row2, k):
    for i in range(k, len(row1)):
        row2[i] ^= row1[i]


# Приводит матрицу к верхнетреугольной форме
def ref(matrix):
    k = -1
    num_rows = len(matrix)
    num_cols = len(matrix[0])

    for j in range(num_cols):
        found_pivot = False
        for i in range(k + 1, num_rows):
            if matrix[i][j] == 1:
                if not found_pivot:
                    k += 1
                    matrix[k], matrix[i] = matrix[i], matrix[k]
                    found_pivot = True
                else:
                    elements_row_xor(matrix[k], matrix[i], j)

    return matrix[:k+1]


# Приводит матрицу к сокращённой ступенчатой форме
def new_rref(matrix, k):
    row = 0
    col = 0
    num_cols = len(matrix[0])

    while row < k and col < num_cols:
        if matrix[row][col] == 1:
            for i in range(row):
                if matrix[i][col] == 1:
                    elements_row_xor(matrix[row], matrix[i], col)

            if col != row:
                matrix[:, row], matrix[:, col] = matrix[:, col].copy(), matrix[:, row].copy()

            row += 1
        col += 1

    return matrix

In [15]:
G = ref(S.copy())
k = len(G)
n = len(G[0])

# Приведенная матрица в ступенчатом виде
G = new_rref(G.copy(), k)
G

array([[1, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 1, 1]])

In [16]:
n, k

(7, 4)

In [17]:
X = G[:,k:].copy()
X

array([[1, 1, 1],
       [1, 1, 0],
       [1, 0, 1],
       [0, 1, 1]])

## Сформировать проверочную матрицу на основе порождающей.

In [18]:
# Создает проверочную матрицу
def createH(X, n, k):
    H = np.zeros((n, n - k), dtype=int)

    H[:k, :] = X.copy()
    H[k:, :] = np.eye(n - k, dtype=int)

    return H

In [19]:
# Проверочная матрица
H = createH(X.copy(), n, k)
H

array([[1, 1, 1],
       [1, 1, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

## Сформировать таблицу синдромов для всех однократных ошибок.

In [20]:
# Создаёт таблицу синдромов
def create_syndrome_table(H, err_count):
    n = len(H)
    one_err = np.eye(n, dtype=int)
    syndrome_table = {str(H[i]): one_err[i] for i in range(n)}

    for num in range(2, err_count + 1):
        syndrome_combinations = np.array(list(iter.combinations(H, num)))
        error_combinations = np.array(list(iter.combinations(one_err, num)))

        syndrome_arr = [np.mod(np.sum(arr, axis=0), 2) for arr in syndrome_combinations]
        err_arr = [np.sum(arr, axis=0) for arr in error_combinations]

        for i in range(len(syndrome_arr)):
            syndrome_table[str(syndrome_arr[i])] = err_arr[i]

    return syndrome_table

In [21]:
syndrome_table = create_syndrome_table(H.copy(), err_count=1)
syndrome_table

{'[1 1 1]': array([1, 0, 0, 0, 0, 0, 0]),
 '[1 1 0]': array([0, 1, 0, 0, 0, 0, 0]),
 '[1 0 1]': array([0, 0, 1, 0, 0, 0, 0]),
 '[0 1 1]': array([0, 0, 0, 1, 0, 0, 0]),
 '[1 0 0]': array([0, 0, 0, 0, 1, 0, 0]),
 '[0 1 0]': array([0, 0, 0, 0, 0, 1, 0]),
 '[0 0 1]': array([0, 0, 0, 0, 0, 0, 1])}

## Сформировать кодовое слово длины n из слова длины k.

In [22]:
word = np.array([1, 0, 0, 1])
code_word = np.dot(word, G) % 2
code_word

array([1, 0, 0, 1, 1, 0, 0])

## Внести однократную ошибку в сформированное слово.

In [23]:
err = np.array([0, 1, 0, 0, 0, 0, 0])
receive_code_word = code_word ^ err
receive_code_word

array([1, 1, 0, 1, 1, 0, 0])

## Вычислить синдром, исправить ошибку с использованием таблицы синдромов.

In [24]:
syndrome = np.dot(receive_code_word, H) % 2
syndrome

array([1, 1, 0])

In [25]:
if not np.array_equal(syndrome, np.zeros(n, dtype=int)):
    receive_code_word ^= syndrome_table[syndrome.__str__()]
receive_code_word

array([1, 0, 0, 1, 1, 0, 0])

## Убедиться в правильности полученного слова.

In [26]:
decoded_word = receive_code_word[:k]
decoded_word

array([1, 0, 0, 1])

In [27]:
np.array_equal(word, decoded_word)

True

## Сформировать кодовое слово длины n из слова длины k.

In [28]:
word = np.array([0, 1, 1, 1])
code_word = np.dot(word, G) % 2
code_word

array([0, 1, 1, 1, 0, 0, 0])

## Внести двукратную ошибку в сформированное слово.

In [29]:
err = np.array([0, 1, 0, 0, 0, 1, 0])
receive_code_word = code_word ^ err
receive_code_word

array([0, 0, 1, 1, 0, 1, 0])

## Вычислить синдром, исправить ошибку с использованием таблицы синдромов.

In [30]:
syndrome = np.dot(receive_code_word, H) % 2
syndrome

array([1, 0, 0])

In [31]:
if not np.array_equal(syndrome, np.zeros(n, dtype=int)):
    receive_code_word ^= syndrome_table[syndrome.__str__()]
receive_code_word

array([0, 0, 1, 1, 1, 1, 0])

## Убедиться, что полученное слово отличается от отправленного.

In [32]:
decoded_word = receive_code_word[:k]
decoded_word

array([0, 0, 1, 1])

In [33]:
np.array_equal(word, decoded_word)

False

# Часть 2.

# Сформировать порождающую матрицу линейного кода (n, k, 5).

In [34]:
k, n = 5, 13
X = np.array([
    [1, 1, 1, 1, 1, 0, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 1],
    [1, 0, 1, 1, 0, 1, 0, 1],
    [0, 1, 1, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 1]
])
G = np.zeros((k,n), dtype=int)
G[:,:k] = np.eye(k, dtype=int)
G[:, k:] = X.copy()
G

array([[1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1]])

# Сформировать проверочную матрицу на основе порождающей.

In [35]:
H = np.zeros((n, n-k), dtype=int)
H[:k, :] = X.copy()
H[k:, :] = np.eye(n-k, dtype=int)
H

array([[1, 1, 1, 1, 1, 0, 0, 1],
       [1, 1, 0, 0, 0, 1, 1, 1],
       [1, 0, 1, 1, 0, 1, 0, 1],
       [0, 1, 1, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

#  Сформировать таблицу синдромов для всех однократных и двукратных ошибок.

In [36]:
syndrome_table = create_syndrome_table(H.copy(), err_count=2)
syndrome_table

{'[1 1 1 1 1 0 0 1]': array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[1 1 0 0 0 1 1 1]': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[1 0 1 1 0 1 0 1]': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[0 1 1 0 1 0 1 1]': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[0 0 0 1 1 1 1 1]': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[1 0 0 0 0 0 0 0]': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 '[0 1 0 0 0 0 0 0]': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 '[0 0 1 0 0 0 0 0]': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
 '[0 0 0 1 0 0 0 0]': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
 '[0 0 0 0 1 0 0 0]': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 '[0 0 0 0 0 1 0 0]': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
 '[0 0 0 0 0 0 1 0]': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
 '[0 0 0 0 0 0 0 1]': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 '[0 0 1 1 1 1 1 0]': array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[0 1 0 0 1 1 0 0]'

# Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [37]:
word = np.array([0, 1, 1, 0, 1])
code_word = np.dot(word, G) % 2
err = np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
receive_code_word = code_word ^ err
syndrome = np.dot(receive_code_word, H) % 2
if not np.array_equal(syndrome, np.zeros(n, dtype=int)):
    receive_code_word ^= syndrome_table[syndrome.__str__()]
decoded_word = receive_code_word[:k]
decoded_word


array([0, 1, 1, 0, 1])

In [38]:
np.array_equal(word, decoded_word)

True

# Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [39]:
word = np.array([1, 1, 0, 0, 1])
code_word = np.dot(word, G) % 2
err = np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0])
receive_code_word = code_word ^ err
syndrome = np.dot(receive_code_word, H) % 2
if not np.array_equal(syndrome, np.zeros(n, dtype=int)):
    receive_code_word ^= syndrome_table[syndrome.__str__()]
decoded_word = receive_code_word[:k]
decoded_word

array([1, 1, 0, 0, 1])

In [40]:
np.array_equal(word, decoded_word)

True

# Сформировать кодовое слово длины n из слова длины k. Внести трёхкратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного.

In [41]:
word = np.array([1, 1, 1, 1, 1])
code_word = np.dot(word, G) % 2
err = np.array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])
receive_code_word = code_word ^ err
syndrome = np.dot(receive_code_word, H) % 2
if not np.array_equal(syndrome, np.zeros(n, dtype=int)):
    receive_code_word ^= syndrome_table[syndrome.__str__()]
decoded_word = receive_code_word[:k]
decoded_word

array([0, 0, 1, 1, 0])

In [42]:
np.array_equal(word, decoded_word)

False